In [20]:
import pandas as pd

In [21]:
# Query used:
"""
CREATE OR REPLACE MODEL
  `scrap_tire_dataset.dc_stc_arima_02` 
  OPTIONS(MODEL_TYPE='ARIMA_PLUS',
    TIME_SERIES_TIMESTAMP_COL='date',
    TIME_SERIES_DATA_COL='total_tires',
    DATA_FREQUENCY = 'DAILY',
    HORIZON = 7,
    CLEAN_SPIKES_AND_DIPS = TRUE,
    ADJUST_STEP_CHANGES = TRUE,
    TREND_SMOOTHING_WINDOW_SIZE = 30,
    TIME_SERIES_ID_COL = ['dc_name', 'size_code'],
    TIME_SERIES_LENGTH_FRACTION =0.5,
    min_time_series_length=30,
    HOLIDAY_REGION='US') AS
SELECT
  date, total_tires, dc_name, size_code
FROM 
  `dulcet-genius-368403.scrap_tire_dataset.dc_tire_collection` ;


"""

"\nCREATE OR REPLACE MODEL\n  `scrap_tire_dataset.dc_stc_arima_02` \n  OPTIONS(MODEL_TYPE='ARIMA_PLUS',\n    TIME_SERIES_TIMESTAMP_COL='date',\n    TIME_SERIES_DATA_COL='total_tires',\n    DATA_FREQUENCY = 'DAILY',\n    HORIZON = 7,\n    CLEAN_SPIKES_AND_DIPS = TRUE,\n    ADJUST_STEP_CHANGES = TRUE,\n    TREND_SMOOTHING_WINDOW_SIZE = 30,\n    TIME_SERIES_ID_COL = ['dc_name', 'size_code'],\n    TIME_SERIES_LENGTH_FRACTION =0.5,\n    min_time_series_length=30,\n    HOLIDAY_REGION='US') AS\nSELECT\n  date, total_tires, dc_name, size_code\nFROM \n  `dulcet-genius-368403.scrap_tire_dataset.dc_tire_collection` ;\n\n\n"

In [27]:
csv_path = '../../data/public/preds/bqml_arima_plus_03'
df_data = pd.read_csv(f"{csv_path}.csv")

In [28]:
df_data['forecast_timestamp'] = pd.to_datetime(df_data['forecast_timestamp'], infer_datetime_format=True)


In [29]:
df_data.dtypes

dc_name                                         object
size_code                                        int64
forecast_timestamp                 datetime64[ns, UTC]
forecast_value                                 float64
standard_error                                 float64
confidence_level                               float64
prediction_interval_lower_bound                float64
prediction_interval_upper_bound                float64
confidence_interval_lower_bound                float64
confidence_interval_upper_bound                float64
dtype: object

In [30]:
def gen_formatted_preds(df_x):
    list_records = []
    for r_index, row in df_x.iterrows():
        dict_record = {
            'date': row['forecast_timestamp'].date(),
            'dc_name': row['dc_name'],
            'size_code': row['size_code'],
            'total_tires': int(max(row['forecast_value'], 0))
        }
        list_records.append(dict_record)
    df_y = pd.DataFrame(list_records)
    display(df_y.head())
    df_y.to_csv(f"{csv_path}_fmt.csv", index=False)

In [31]:
gen_formatted_preds(df_data)

,date,dc_name,size_code,total_tires
0,2022-09-20,BAKERSFIELD,1856015,6
1,2022-09-21,BAKERSFIELD,1856015,12
2,2022-09-22,BAKERSFIELD,1856015,7
3,2022-09-23,BAKERSFIELD,1856015,14
4,2022-09-24,BAKERSFIELD,1856015,17
